_Softwarepraktikum Blockkurs März 2021 (WS2020/21)_ 

# Fortgeschrittene Symbolische Programmierung I[<sup>1</sup>](#fn1)

[Jörn Behrens](https://www.math.uni-hamburg.de/numgeo/) (joern.behrens@uni-hamburg.de)

## 1. Einführung und Ziel

Nachdem wir nun erste Erfahrungen mit der symbolischen Programmierung gesammelt haben und nachdem wir auch Graphiken erzeugen können, werden wir uns nun etwas tiefer in die symbolische Programmierung mit SymPy einarbeiten. 

## 2. Spezifika und Fallstricke

### Expression-Trees

Zunächst brauchen wir ein erweitertes Verständnis davon, wie SymPy arbeitet, damit wir manche der Reaktionen verstehen. Dazu beginnen wir mit der Einführung von sogenantnen _Expression-Trees_. Jeder symbolische Ausdruck in SymPy ist zerlegt in elementare Ausdrück, die wiederum hierarchisch aufgebaut sind. Beispiele in diesem Teil sind angelehnt an die [SymPy Dokumentation](https://docs.sympy.org/latest/tutorial/manipulation.html).

In [ ]:
from sympy import *

x,y,z = symbols('x y z')
expr  = x**2 + x*y
srepr(expr)

Im obigen Beispiel haben wir zunächst einen Ausdruck $x^2+xy$ erzeugt und mit dem Befehl `srepr` analysiert. Die Klammerschachtelung entspricht dabei einer Baumtiefe. Der zugehörige Expression-Tree sieht dann etwa so aus:
![Expression Tree 1](figs/SWPrak07-exprtree1.png "Expression Tree for x**2 + x*y")

Interessant sind die Expression-Trees für die Operationen $x-y$ und $\frac{x}{y}$:

In [ ]:
srepr(x-y)

Dies entspricht dem folgenden Expression-Tree
![Expression Tree 2](figs/SWPrak07-exprtree2.png "Expression Tree for x-y")

In [ ]:
srepr(x/y)

Dies entspricht dem folgenden Expression-Tree
![Expression Tree 3](figs/SWPrak07-exprtree3.png "Expression Tree for x/y")

Es gibt also keine Division und auch keine Subtraktion, sondern lediglich die Multiplikation und die Addition, sowie die Potenz.

Häufig wird auch die Reihenfolge der Operationen in der internen Darstellung vertauscht, wenn angenommen werden kann, dass die Operationen kommutativ sind. Soll das ausgeschlossen werden, kann ein Attribut bei der Definition des Symbols entsprechend gesetzt werden, z.b. `Symbol('A',commutative=false)`.

In [ ]:
srepr(-y+x)

### Die func Methode

Jedes Objekt in SymPy besitzt eine Methode `func`, welche die jeweilige elementare Funktion, die sich hinter der symbolischen Schreibweise verbirgt, anzeigt. Hier ein paar Beispiele. Was verbirgt sich hinter der Addition einer symbolischen Variable x mit sich selbst, also $x+x$?

In [ ]:
expr = Add(x,x)
expr.func

Zu unserer Überraschung verbirgt sich hinter der Addition von x mit sich selbst eine Multiplikation! Wir stellen fest, dass `Add(x,x)` automatisch in eine Multiplikation von $x$ mit 2, also in $2x$ übersetzt wird:

In [ ]:
expr

Weitere einfache Beispiele:

In [ ]:
Integer(2).func

In [ ]:
Integer(0).func

In [ ]:
Integer(-1).func

### Die args Methode

So, wie es eine `func` Methode gibt, so gibt es für jedes Symbol auch eine `args` Methode, welche die Argumente angibt, die in die elementare Funktion übergeben werden. 

**Die Argumentenliste ist leer, wenn wir an einem Blatt des Expression-Tree angekommen sind!**

Schauen wir uns das einmal an einem Beispiel ($3y^2x$) an:

In [ ]:
expr = 3 * y**2 * x
expr.func

In [ ]:
expr.args

In [ ]:
Integer(3).args

In [ ]:
x.args

Der gesamte Ausdruck kann aus den beiden Methoden `func` und `args` rekonstruiert werden!

_Bemerkung: Wir müssen für die Argumentliste `*expr.args` angeben, damit der die Argumentliste in der Funktion ausgewerten werden kann, eigentlich wird hier ein Zeiger übergeben und nicht die Liste selbst_

In [ ]:
expr.func(*expr.args)

Wir sehen also, dass der folgende Ausdruck wahr ist:

In [ ]:
expr == expr.func(*expr.args)

Wir können eine Invariante (oder einen Satz) formulieren:

>**Satz: Für jeden wohl-formulierten SymPy Ausdruck `expr` gilt entweder**
>* **`expr.args = ()`, oder**
>* **`expr = expr.func(*expr.args)`**

### Baum-Traversierung

Mit diesen Methoden können wir nun ein kleines rekursives Programm schreiben, mit dem der Expression-Tree durchlaufen werden kann. Wir wissen, dass ein Blatt erreicht ist, wenn die Argumentenliste leer ist und können damit den rekursiven Algorithmus abbrechen lassen. 

Wir wollen uns auf jeder Ebene den Teilbaum-Ausdruck ausgeben lassen. Das Eingabe-Argument für unseren Algorithmus ist also ein SymPy Ausdruck `expr` von welchem wir uns zunächst einen Eindruck verschaffen, bevor wir die Argumente jeweils wieder als Ausdrücke rekursiv weitergeben.

In [ ]:
def rec_travers(expr):
    print(expr)
    for arg in expr.args:
        rec_travers(arg)

Wenden wir nun dieses kleine Programm auf einen Ausdruck an, zum Beispiel $x*y + 1$:

In [ ]:
expr = x*y + 1
rec_travers(expr)

_Bemerkung: Diese Art der Traversierung ist schon als eine der SymPy Routinen vorhanden. Es gibt dazu die Funktionen `preorder_traversal` und `postorder_traversal`._

### Auswertung verhindern

Manchmal ist es sinnvoll die automatische Auswertung bzw. Umwandlung von Ausdrücken zu verhindern. Wir haben oben gesehen, dass $x+x$ automatisch zu $2x$ wird. Wenn man das vermeiden möchte, gibt es verschiedene Möglichkeiten:

* `expr = Add(x,x,evaluate=false)`
* `sympify("x + x",evaluate=false)`
* `expr = x + UnevaluatedExpr(x)`

In [ ]:
expr1 = x+x
expr2 = Add(x,x,evaluate=false)
expr3 = sympify("x+x",evaluate=false)
expr4 = x + UnevaluatedExpr(x)
print(expr1,'  ;  ',expr2,'  ;  ',expr3,'  ;  ',expr4)

### Gleichheit

Das Gleichheitszeichen ist eine Zuordnung keine Gleichung! So können Ausdrücke zugeordnet werden

In [ ]:
a = (x + y)**2
print(a)

Um Gleichheit von Ausdrücken zu testen, kann das doppelte Gleicheitszeichen `==` verwendet werden. Es kann jedoch nicht gleichzeitig Vereinfachen, sondern testet tatsächlich exakte Gleichheit von Ausdrücken:

In [ ]:
(x + y)**2 == x**2 + 2*x*y + y**2

In [ ]:
(x + y)**2 == (x + y)**2

Dabei heißt exakte Gleichheit, eben im Sinne des Expression-Trees. Die folgenden Ausdrücke sind natürlich exakt gleich, denn sie werden intern in exakt derselben Weise dargestellt:

In [ ]:
3 * y**2 * x == x * 3 * y**2

In [ ]:
srepr(3 * y**2 * x)

In [ ]:
srepr(x * 3 * y**2)

Um symbolische Gleichheit zu testen, kann man die zu testenden Ausdrücke von einander subtrahieren und in einer Funktion wie `simplify`, `expand`, etc. testen:

In [ ]:
simplify((x+y)**2 - (x**2 + 2*x*y + y**2))

### Variablen

**Achtung:** Variablen behalten zunächst immer den Wert, der ihnen bei Instantiierung zugewiesen wurde. Hier ein Beispiel:

In [ ]:
a = Symbol('a')
print(a)
b = a + 1
print(b)
a = 4
print(a)
print(b)

Wenn aber die Variable auch neu berechnet wird, dann kann auch ihr Wert aufdatiert werden. Das folgende Beispiel zeigt es:

In [ ]:
p, z, s = var('preis zahl summe')
d = p*z
print('Original d: ',d)
p = 5
z = 40
print('p: ',p,' z: ',z,' d: ',d)
d = p*z
print('d nach Neuberechnung: ',d)

Eine elegante Lösung ist es, sich entsprechende Funktionen einzurichten:

In [ ]:
def sumpreis():
    return p*z

In [ ]:
p , z = var('preis zahl')
print(sumpreis())
p = 5
z = 40
print(sumpreis())

### Symbole und Namen

Bevor man eine Variable oder ein Symbol nutzen kann muss es natürlich deklariert werden. Der folgende Ausdruck wirft daher eine Fehlermeldung:

In [ ]:
expr = u**2

Nach einer Deklaration, die mit einem der folgenden Befehle erfolgen kann, löst man das Problem.
* `sympy.var('u')` oder einfach nur `var('u')`
* `u = symbols('u')`

In [ ]:
var('u')
expr = u**2
print(expr)

Die Liste der vordeklarierten Symbole und Funktionen kann man sich mit dem folgenden Befehl anzeigen lassen. Vorsicht, die Liste ist lang!

In [ ]:
import sympy
dir(sympy)

**Warnung:** Python (und damit auch SymPy) erlaubt es, Symbole zu überschreiben. Das können wir am folgenden Beispiel demonstrieren, in dem wir einfach die Konstante $\pi$ und die Cosinus-Funktion $\cos$ durch Unsinn ersetzen. Glücklicherweise kann der Standard einfach wieder hergestellt werden, indem die entsprechenden Symbole erneut importiert werden.

In [ ]:
print('cos(pi) original: ',cos(pi))
pi = 4
def cos(x):
    return 2*x
print('cos(pi) nach Überschreiben: ',cos(pi))
from sympy import cos, pi
print('cos(pi) nach Wiederherstellung: ',cos(pi))

### Zahldarstellung

Fließkommazahlen können in SymPy mit dem Befehl `Float` erzeugt werden. Dabei kann man die Anzahl der signifikanten Ziffern angeben:

In [ ]:
print('Default Darstellung mit 15 Ziffern: ',Float(pi))
print('Darstellung mit 25 Ziffern: ', Float(pi,25))

Rationale Zahlen lassen sich mit dem Befehl `Rational` erzeugen. Hier ein paar Beispiele:

In [ ]:
list = [Rational(1,7), Rational(1,4), Rational(1,100)]
list

Jetzt schauen wir, wie sich diese Zahlen als Fließkommazahlen darstellen! Wir wollen dabei jeweils die 15 Ziffern sehen

In [ ]:
for r in list:
    print(Float(r))

Die Genauigkeit der Fließkommazahlen ist jedoch nicht immer beliebig. In dem Moment, in dem eine Fließkommazahl angelegt wird, wird sie als Binärzahl gespeichert. Will man mehr Ziffern anzeigen, so werden diese im Zweifel mit (binären) Nullen erzeugt, was die Genauigkeit nicht erhöht. Hier ein Beispiel mit der Rationalen Zahl $\frac{1}{7}$, die sich nicht als exakte Fließkommazahl (und schon gar nicht als Binärzahl) darstellen lässt.

In [ ]:
einsiebtel_rational = Rational(1,7)
einsiebtel_float3   = Float(einsiebtel_rational,3)
print(einsiebtel_float3)
einsiebtel_floatf25 = Float(einsiebtel_float3,25)
print(einsiebtel_floatf25)
einsiebtel_floatt25 = Float(einsiebtel_rational,25)
print(einsiebtel_floatt25)

## 3. Aufgaben

1. **Aufgabe (Fließkommadarstellung)**: Schätzen Sie zunächst das Ergebnis der SymPy operation `Float('0.1',10) + Float('0.1',3)`. Implementieren Sie diese Operation und versuchen Sie das Ergebnis zu interpretieren.
1. **Aufgabe (Expression-Tree)**: Erzeugen Sie einen Expression-Tree für die folgenden Ausdrücke:
 * $\int_0^{\pi} \cos(2x)^2 - x^2\ dx$
 * $\sqrt{2\sin(x^2)}$
1. **Aufgabe (Traversierung)**: Traversieren Sie die beiden Ausdrücke aus der vorigen Aufgabe mit Hilfe des kleinen rekursiven Programms.

---
1) <span id="fn1">Copyright Notice:
    
    Fortgeschrittene Symbolische Programmierung I, Copyright (C) 2020  Jörn Behrens
    
        Prof. Dr. Jörn Behrens
        Universität Hamburg, Dept. Mathematik
        Bundesstrasse 55
        20146 Hamburg, Germany
        joern.behrens@uni-hamburg.de

    This program is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License along
    with this program; if not, write to the Free Software Foundation, Inc.,
    51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.


</span>

In [ ]:
help(cos)